<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data_last_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
Data_basic = pd.DataFrame()

In [4]:
Data_basic = pd.DataFrame(columns=[
    'GameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2021,2023))

In [5]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [6]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [7]:
#Data_basic.to_csv('/content/drive/MyDrive/play_ID_data.csv', index=False)


In [8]:
print(Data_basic)





        GameID  season  types  week HomeTeam_ID AwayTeam_ID Winner
0    401326322    2021      2     1          27           6   Home
1    401326315    2021      2     1           1          21   Away
2    401326308    2021      2     1           2          23   Away
3    401326316    2021      2     1           4          16   Home
4    401326317    2021      2     1           8          25   Away
..         ...     ...    ...   ...         ...         ...    ...
539  401437957    2022      2    18          21          19   Home
540  401437962    2022      2    18          25          22   Home
541  401437963    2022      2    18          26          14   Home
542  401437959    2022      2    18          28           6   Home
543  401437952    2022      2    18           9           8   Away

[544 rows x 7 columns]


In [41]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','clock','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'Penalty_Home_yards',
  'Penalty_Away_yards',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_yards_home',
  'sack_yards_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away'])

In [42]:
def stats(GameID,Quarter,time):
  url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
  response_drive = r.get(url)
  drive_data = response_drive.json()
  drive_df = json_normalize(drive_data['items'])
  data=[]
  time = datetime.timedelta(seconds = time)
  clock = datetime.timedelta(seconds = 0)
  Pass_Incompletion_Home=0
  Punt_Home=0
  Timeout_Home=0
  Punt_Away=0
  Pass_Incompletion_Away=0
  Timeout_Away=0
  homeScore=0
  awayScore=0
  PassReception_Home_yards=0
  PassReception_Away_yards=0
  Penalty_Home=0
  Penalty_Away=0
  Penalty_Home_yards=0
  Penalty_Away_yards=0
  rushing_yards_home=0
  rushing_yards_away=0
  field_goals_good_home=0
  field_goals_good_away=0
  field_goals_missed_home=0
  field_goals_missed_away=0
  fumble_recovery_oponnent_home=0
  fumble_recovery_oponnent_away=0
  fumble_recovery_own_home=0
  fumble_recovery_own_away=0
  sack_yards_home=0
  sack_yards_away=0
  passing_TD_home=0
  passing_TD_away=0
  rushing_TD_home=0
  rushing_TD_away=0
  interception_home=0
  interception_away=0
  Interception_Return_TD_home=0
  Interception_Return_TD_away=0
  kick_off_return_TD_home=0
  kick_off_return_TD_away=0
  safety_home=0
  safety_away=0
  for index, row in drive_df.iterrows():
    try:
      if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
        for detail in range(len(row['plays.items'])):
          duration_str = row['plays.items'][detail]['clock']['displayValue']
          duration_arr = duration_str.split(":")
          duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
          clock = datetime.timedelta(seconds = 900) - duration
          if row['plays.items'][detail]['period']['number'] <= Quarter and clock <= time:
            if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
              Pass_Incompletion_Home=Pass_Incompletion_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
              Punt_Home=Punt_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
              Timeout_Home=Timeout_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
              PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
              Penalty_Home_yards=Penalty_Home_yards+row['plays.items'][detail]['statYardage']
              Penalty_Home=Penalty_Home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
              rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
              sack_yards_home=row['plays.items'][detail]['statYardage']+sack_yards_home
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
              passing_TD_home=passing_TD_home+1
              PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
              rushing_TD_home=rushing_TD_home+1
              rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
              interception_home=interception_home + 1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
              field_goals_good_home=field_goals_good_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
              field_goals_missed_home=field_goals_missed_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
              Interception_Return_TD_home=Interception_Return_TD_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
              kick_off_return_TD_home=kick_off_return_TD_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
              fumble_recovery_own_home=fumble_recovery_own_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
              fumble_recovery_oponnent_home=fumble_recovery_oponnent_home+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
              safety_home=safety_home+1
            homeScore=row['plays.items'][detail]['homeScore']
      elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
        for detail in range(len(row['plays.items'])):
          duration_str2 = row['plays.items'][detail]['clock']['displayValue']
          duration_arr2 = duration_str2.split(":")
          duration2 = datetime.timedelta(minutes=int(duration_arr2[0]), seconds=int(duration_arr2[1]))
          clock = datetime.timedelta(seconds = 900) - duration2
          if row['plays.items'][detail]['period']['number'] <= Quarter and clock <= time:
            if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
              Pass_Incompletion_Away=Pass_Incompletion_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
              Punt_Away=Punt_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
              Timeout_Away=Timeout_Away+1  
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
              PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
              Penalty_Away_yards=Penalty_Away_yards+row['plays.items'][detail]['statYardage']
              Penalty_Away=Penalty_Away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
              rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
              sack_yards_away=row['plays.items'][detail]['statYardage']+sack_yards_away
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
              passing_TD_away=passing_TD_away+1
              PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
              rushing_TD_away=rushing_TD_away+1
              rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
              interception_away=interception_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
              field_goals_good_away=field_goals_good_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
              field_goals_missed_away=field_goals_missed_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
              Interception_Return_TD_away=Interception_Return_TD_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
              kick_off_return_TD_away=kick_off_return_TD_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
              fumble_recovery_own_away=fumble_recovery_own_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
              fumble_recovery_oponnent_away=fumble_recovery_oponnent_away+1
            elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
              safety_away=safety_away+1
            awayScore=row['plays.items'][detail]['awayScore']
    except TypeError:
      print('it doenst worked')
  data = [    GameID,    clock, Pass_Incompletion_Home,
          Punt_Home,    Timeout_Home,
          Punt_Away,    Pass_Incompletion_Away,
          Timeout_Away,    homeScore,
          awayScore,    PassReception_Home_yards,
          PassReception_Away_yards,    Penalty_Home,
          Penalty_Away,    Penalty_Home_yards,
          Penalty_Away_yards,    rushing_yards_home,
          rushing_yards_away,    field_goals_good_home,
          field_goals_good_away,    field_goals_missed_home,
          field_goals_missed_away,    fumble_recovery_oponnent_home,
          fumble_recovery_oponnent_away,    fumble_recovery_own_home,
          fumble_recovery_own_away,    sack_yards_home,
          sack_yards_away,    passing_TD_home,
          passing_TD_away,    rushing_TD_home,
          rushing_TD_away,    interception_home,
          interception_away,    Interception_Return_TD_home,
          Interception_Return_TD_away,    kick_off_return_TD_home,
          kick_off_return_TD_away,
          safety_home,    safety_away]
  stats_df.loc[len(stats_df)] = data




In [40]:
GameID='401326322'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0
for index, row in drive_df.iterrows():
        for detail in range(len(row['plays.items'])):
          #print(detail)
          print(row['plays.items'][detail]['type']['text'])
          print(row['plays.items'][detail]['statYardage'])


Kickoff
0
Rush
5
Rush
3
Pass Incompletion
0
Punt
0
Pass Reception
28
Rush
4
Pass Reception
5
Rush
2
Rush
4
Pass Reception
16
Pass Incompletion
0
Pass Incompletion
0
Penalty
5
Pass Incompletion
0
Punt
0
Rush
5
Pass Reception
10
Pass Incompletion
0
Pass Reception
19
Pass Reception
28
Pass Reception
16
Rush
3
Pass Reception
8
Passing Touchdown
5
Kickoff
0
Pass Reception
11
Pass Reception
8
Pass Reception
12
Pass Reception
7
Pass Reception
8
Pass Reception
7
Passing Touchdown
22
Kickoff
0
Rush
3
Pass Reception
27
Rush
5
Pass Incompletion
0
Pass Incompletion
0
End Period
0
Punt
-10
Punt
2
Pass Reception
2
Pass Reception
5
Rush
0
Penalty
-5
Punt
14
Rush
2
Pass Reception
5
Pass Reception
19
Pass Reception
17
Rush
5
Passing Touchdown
2
Kickoff
0
Pass Reception
22
Pass Incompletion
14
Pass Reception
6
Pass Reception
8
Pass Incompletion
-5
Pass Reception
3
Pass Reception
4
Pass Incompletion
0
Field Goal Missed
31
Fumble Recovery (Opponent)
1
Rush
2
Pass Incompletion
0
Timeout
0
Pass Reception
13

In [12]:
for plays in Data_basic['GameID']:
  stats(plays,1,900)
  

it doenst worked


KeyboardInterrupt: ignored

In [ ]:
#stats_df.to_csv('/content/drive/MyDrive/quarter1_data.csv', index=False)

In [43]:
stats('401326322',4,900)

In [44]:
stats_df

,GameID,clock,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,...,rushing_TD_home,rushing_TD_away,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away,safety_home,safety_away
0,401326322,0 days 00:14:58,16,5,1,3,18,5,31,29,...,0,0,2,1,0,0,0,0,0,0


In [ ]:

url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401326316/competitions/401326316/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
 



In [ ]:
import datetime

duration_str = "15:00"
duration_arr = duration_str.split(":")
duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))

print(duration)

In [19]:
GameID='401326322'
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    #print(row['plays.items'][detail])

    duration_str = row['plays.items'][detail]['clock']['displayValue']
    duration_arr = duration_str.split(":")
    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
    print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
    if (datetime.timedelta(seconds=900)-duration) <= (datetime.timedelta(seconds=60)):
      print(datetime.timedelta(seconds=900)-duration)

27
0:00:00
6
0:00:00
6
0:00:44
6
6
6
6
27
27
27
27
27
27
27
27
27
6
27
27
27
6
27
27
6
27
6
6
6
6
27
6
27
6
27
27
27
27
27
6
27
27
0:00:00
6
0:00:07
27
0:00:18
6
0:00:41
27
6
6
27
27
27
27
6
27
6
6
6
6
6
27
6
6
6
27
6
6
6
6
6
6
6
27
27
27
6
6
6
27
27
27
6
6
6
6
6
6
6
6
6
6
27
6
27
27
27
27
27
6
0:00:00
6
0:00:00
6
0:00:03
6
0:00:33
6
0:00:57
6
6
6
6
6
6
6
6
6
6
27
27
27
27
6
6
27
27
6
27
27
6
6
6
6
6
6
6
6
6
6
6
27
27
27
27
27
0:00:00
27
0:00:40
27
27
27
6
6
6
6
6
27
27
27
27
27
27
27
27
27
27
27
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
27
27
27
27
27
27
27
27
27
27
27
27
27
27


In [ ]:
print(duration-datetime.timedelta(seconds=110))

In [ ]:
15*60